In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns

In [ ]:
df = pd.read_csv("C:/Users/danie/Downloads/SmA-Four-Tank-Batch-Process_V2.csv", sep = ";")

In [ ]:
col = []

for x in df.columns:
    col.append(x.split(" ")[0])
    
df.columns = col

In [ ]:
# Entferne alle Schritte mit StepID = 2, da diese für das Projekt nicht relevant sind
df.drop(df[df["CuStepNo"] == 2].index, inplace = True)

In [ ]:
# Extrahiere die Zeitangaben bzw. formatiere die Zeitstempel in das richtige Format
df.timestamp = pd.to_datetime(df.timestamp)

# df['timestamp']=pd.to_datetime(df['timestamp'],format='%Y-%m-%dT%H:%M:%S.%f')

df["day"] = df.timestamp.dt.day
df["hour"] = df.timestamp.dt.hour
df["minute"] = df.timestamp.dt.minute
df["second"] = df.timestamp.dt.second

In [ ]:
df.columns

In [ ]:
df[df.any(axis = 1).isna()]

In [ ]:
# determine start and end of steps
df['dstep_p']=df['CuStepNo'].diff()
df['dstep_n']=df['CuStepNo'].diff(-1)

vsteps = [1,7,8,3]

# select rows with a step change
dfsen=df[(df['dstep_n']!=0)]
dfsep=df[(df['dstep_p']!=0)]
dfse=pd.concat([dfsen,dfsep])
dfse=dfse.sort_values(by=['timestamp'])

# create new dataframe where we store extracted information
dfinfo_steps=pd.DataFrame(columns=['step_length','start','end','stepn'])

# iterative approach
pstep=-1
c=0
for n in range(dfse.shape[0]):
    # get row
    r=dfse.iloc[n]
    if pstep==r['CuStepNo']:
        # determine step length
        stepl=r['timestamp']-dfse.iloc[n-1]['timestamp']
        # update dataframe
        dfinfo_steps.loc[c]=(stepl,dfse.iloc[n-1]['timestamp'],r['timestamp'],r['CuStepNo'])
        c=c+1
    else:
        pstep=r['CuStepNo']
print(dfinfo_steps.head())
print('Max step_length: {}'.format(dfinfo_steps['step_length'].max()))
print('Min step_length: {}'.format(dfinfo_steps['step_length'].min()))
print('#steps: {}'.format(dfinfo_steps.shape[0]))

# now determine whether the batch is complete
batchn=1
batchi=-1
dfinfo_steps["batchn"]=0
dfinfo_steps["is_complete"]=False
dfinfo_batches=pd.DataFrame(columns=['batch_length','start','end','steps','batchn','is_complete'])
n=0
b=0
while True:
    if n+len(vsteps)>dfinfo_steps.shape[0]:
        # complete info at incomplete, last batch
        steps=[]
        for v in range(dfinfo_steps.shape[0]-n):
            dfinfo_steps.at[n+v,'batchn']=batchi
            dfinfo_steps.at[n+v,'is_complete']=False
            steps.append(dfinfo_steps.at[n+v,'stepn'])
        dfinfo_batches.loc[b]=[dfinfo_steps.at[n+v,'end']-dfinfo_steps.at[n,'start'],dfinfo_steps.at[n,'start'], \
                               dfinfo_steps.at[n+v,'end'],steps,batchi,False]
        b=b+1
        break
    # check if all steps of a batch are present and in correct order
    isCorrect=True
    for v in range(len(vsteps)):
        isCorrect=dfinfo_steps.loc[n+v,'stepn']==vsteps[v]
        if not isCorrect:
            break
    if isCorrect:
        steps=[]
        for v in range(len(vsteps)):
            dfinfo_steps.at[n+v,'batchn']=batchn
            dfinfo_steps.at[n+v,'is_complete']=True
            steps.append(dfinfo_steps.at[n+v,'stepn'])
        dfinfo_batches.loc[b]=[dfinfo_steps.at[n+v,'end']-dfinfo_steps.at[n,'start'],dfinfo_steps.at[n,'start'], \
                               dfinfo_steps.at[n+v,'end'],steps,batchn,True]
        n=n+len(vsteps)
        batchn=batchn+1
        b=b+1
    else:
        steps=[]
        for vc in range(v):
            dfinfo_steps.at[n+vc,'batchn']=batchi
            dfinfo_steps.at[n+vc,'is_complete']=False
            steps.append(dfinfo_steps.at[n+v,'stepn'])
        dfinfo_batches.loc[b]=[dfinfo_steps.at[n+vc,'end']-dfinfo_steps.at[n,'start'],dfinfo_steps.at[n,'start'], \
                               dfinfo_steps.at[n+vc,'end'],steps,batchi,False]
        batchi=batchi-1
        n=n+vc
        b=b+1
# save dfinfo_steps to file
dfinfo_steps.to_pickle('SmA-Four-Tank-Info-Steps.pkl')
dfinfo_batches.to_pickle('SmA-Four-Tank-Info-Batches.pkl')


In [ ]:
df_steps = pd.read_pickle("C:/Users/danie/SmA-Four-Tank-Info-Steps.pkl")
df_batches = pd.read_pickle("C:/Users/danie/SmA-Four-Tank-Info-Batches.pkl")

In [ ]:
df_batches

In [ ]:
# Entferne den letzten unvollständigen Batch aus dem Datensatz

df = df.drop(df[df.timestamp >= '2018-10-31 14:29:32'].index)

In [ ]:
df[df.DeviationID == 0]

In [ ]:
# Entferne einen Zeileneintrag, bei dem die DeviationID 0 ist

df = df.drop(df[df.DeviationID == 0].index)

In [ ]:
df.columns.sort_values()

In [ ]:
components = ["PI10003", "YS10004", "LI10002", "VE1000", "YC10001", "PI12002", "PL1100", "PL1200", "PI12003", "YC14001", "FIC14002",
"YC21001", "FIC21002", "LIC21002", "VE2100", "YS21004", "FIC22002", "YC2201", "LIC22002", "VE2200", "YS22004",
"FIC23002", "YC23001", "LIC23002", "VE2300", "YS23004", "PI14012", "PI14013", "PIC14007", "YC14008", "YC21006", "YC22006", "YC23006",
"PL2150", "YS14004", "YS14005"]

In [ ]:
df.columns.sort_values()

In [ ]:
#px.line(df[df.DeviationID > 1][["timestamp", "FIC14002_MV", "FIC14002_PV_Out", "FIC14002_SP", "DeviationID"]].drop_duplicates("timestamp"), x = "timestamp",
 #      y = ["FIC14002_MV", "FIC14002_PV_Out", "FIC14002_SP"])

In [ ]:
df.columns

In [ ]:
# DevId
# 1 --> Pl1200 (zwischen PL12002 und PL12003)
# 2 --> PL1100 (zwischen YC10001 und YC14001)
# 3 --> YC21006 (zwischen )
# 4 --> YC22006
# 5 --> YC23006
# 6 --> YS14005
# 7 --> Pl2150
# 8 --> YS14004
# 9 --> YS10004

In [ ]:
df.groupby("CuStepNo").agg({"second":"sum"})

In [ ]:
# Wochenenden am:
# 13.-14.
# 20.-21.
# 27.-28.

In [ ]:
df[df.DeviationID > 1].day.unique()
# 16. und 23. Oktober komplett fehlerfrei

In [ ]:
px.scatter(df[df.DeviationID > 1].sort_values(by = "DeviationID"), x = "timestamp", y = "DeviationID", 
           facet_row = "CuStepNo")

In [ ]:
px.histogram(df.groupby(["DeviationID", "CuStepNo"], as_index = False).agg({"second":"sum"}),
      x = "CuStepNo", y = "second", facet_row = "DeviationID", height = 1000,
      title = "In welchem Prozessschritt hält ein Fehler am längsten an?")

In [ ]:
df.day.unique()

In [ ]:
df_steps.sort_values("step_length", ascending = False)